In [1]:
import bacteriopop_utils
import feature_selection_utils
import load_data

In [2]:
loaded_data = load_data.load_data()

In [3]:
loaded_data.shape

(64755, 11)

In [4]:
loaded_data.head()

,kingdom,phylum,class,order,family,genus,length,oxygen,replicate,week,abundance
sampleID,,,,,,,,,,,
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylobacter,9948861,Low,1,4,0.228531
1056013,Bacteria,Proteobacteria,Betaproteobacteria,Methylophilales,Methylophilaceae,Methylotenera,5066955,Low,1,4,0.220860
1056013,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,4654774,Low,1,4,0.054719
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,,3046340,Low,1,4,0.047956
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,,,,5620690,Low,1,4,0.040903


How many taxa are in each groupby?

In [38]:
# df[['col1', 'col2', 'col3', 'col4']].groupby(['col1', 'col2']).agg(['mean', 'count'])
taxa_per_sample = loaded_data.groupby(['week', 'oxygen', 'replicate'])['abundance'].agg('count')

In [40]:
taxa_per_sample.head(20)

week  oxygen  replicate
4     High    1            877
              2            751
              3            931
              4            748
      Low     1            791
              2            833
              3            803
              4            766
5     High    1            931
              2            728
              3            635
              4            928
      Low     1            851
              2            735
              3            786
              4            684
6     High    1            911
              2            642
              3            791
              4            808
Name: abundance, dtype: int64

## Check that the abundances sum to 1 as expected on this raw data. 

In [5]:
abs(-0.01)

0.01

In [6]:
sample_abundance_sums = loaded_data.groupby(
    ['oxygen', 'replicate', 'week'])['abundance'].sum()

In [7]:
# Look for values that aren't really close to 1.
print [v for v in sample_abundance_sums.unique() if v <0.999]
# an empty list is good!

[]


implement as a function that returns true if the appropriate groupby objects have abundances sum to 1. 

In [8]:
bacteriopop_utils.check_abundances_sums(loaded_data)

all groups of oxygen/week/replicate have abundances that sum to 1


True

Make a negative control by deleting the first three rows of the dataframe.

In [9]:
loaded_data.head()

,kingdom,phylum,class,order,family,genus,length,oxygen,replicate,week,abundance
sampleID,,,,,,,,,,,
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylobacter,9948861,Low,1,4,0.228531
1056013,Bacteria,Proteobacteria,Betaproteobacteria,Methylophilales,Methylophilaceae,Methylotenera,5066955,Low,1,4,0.220860
1056013,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,4654774,Low,1,4,0.054719
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,,3046340,Low,1,4,0.047956
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,,,,5620690,Low,1,4,0.040903


In [10]:
broken_data = loaded_data.copy()
broken_data = broken_data.iloc[3:]

In [11]:
bacteriopop_utils.check_abundances_sums(broken_data)

number of samples that don't sum to 1: 1


False

In [12]:
loaded_data.shape

(64755, 11)

In [13]:
broken_data.shape

(64752, 11)

## On to the new functions

In [14]:
apl = bacteriopop_utils.aggregate_on_phylo_level(loaded_data, 'order')

In [15]:
apl.head()

abundance
oxygen replicate week kingdom phylum        class        order                       
High   1         4    Archaea                                                0.000028
                              Crenarchaeota Thermoprotei                     0.000026
                                                         Acidilobales        0.000007
                                                         Desulfurococcales   0.000005
                                                         Sulfolobales        0.000015

In [16]:
bacteriopop_utils.check_abundances_sums(apl)

all groups of oxygen/week/replicate have abundances that sum to 1


True

In [17]:
fba = bacteriopop_utils.filter_by_abundance(loaded_data, low = 0.01)
fba.head()

first (up to) 5 phylo columns to 
keep: ['Methylobacter' 'Methylotenera' 'Flavobacterium' '' 'Acidovorax']


,kingdom,phylum,class,order,family,genus,oxygen,replicate,week,abundance
sampleID,,,,,,,,,,
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,Methylobacter,Low,1,4,0.228531
1056013,Bacteria,Proteobacteria,Betaproteobacteria,Methylophilales,Methylophilaceae,Methylotenera,Low,1,4,0.220860
1056013,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Low,1,4,0.054719
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,Methylococcales,Methylococcaceae,,Low,1,4,0.047956
1056013,Bacteria,Proteobacteria,Gammaproteobacteria,,,,Low,1,4,0.040903


In [18]:
bacteriopop_utils.check_abundances_sums(fba)

number of samples that don't sum to 1: 88


False

*** We expected `filter_by_abundance()` to cause our abundances to sum to a number less than 1 ***

make sure "other" which is now called "" is there.

In [19]:
fba['genus'].unique()

array(['Methylobacter', 'Methylotenera', 'Flavobacterium', '',
       'Acidovorax', 'Dechloromonas', 'Methylomonas', 'Methylosarcina',
       'Methylomicrobium', 'Methylovulum', 'Methyloglobulus',
       'Pseudomonas', 'Sorangium', 'Burkholderia', 'Polaromonas',
       'Methylovorus', 'Streptomyces', 'Methylophilus', 'Curvibacter',
       'Comamonas', 'Mycobacterium', 'Desulfobacter', 'Rheinheimera',
       'Azospirillum', 'Hylemonella', 'Bdellovibrio', 'Delftia',
       'Lactobacillus', 'Flectobacillus', 'Bosea', 'Bacteriovorax'], dtype=object)

## `reduce_data()` puts together the two functions above.

In [20]:
rd_order = bacteriopop_utils.reduce_data(dataframe = loaded_data, 
                                   min_abundance = 0.05,
                                  phylo_column='order')
rd_order.head(15)

all groups of oxygen/week/replicate have abundances that sum to 1
columns after aggregating on phylo level: Index([u'oxygen', u'replicate', u'week', u'kingdom', u'phylum', u'class',
       u'order', u'abundance'],
      dtype='object')
first (up to) 5 phylo columns to 
keep: ['Flavobacteriales' 'Burkholderiales' 'Methylophilales' 'Methylococcales'
 'Actinomycetales']


,oxygen,replicate,week,kingdom,phylum,class,order,abundance
0,High,1,4,Archaea,,,,0.000028
1,High,1,4,Archaea,Crenarchaeota,Thermoprotei,,0.000026
6,High,1,4,Archaea,Euryarchaeota,,,0.000032
8,High,1,4,Archaea,Euryarchaeota,Halobacteria,,0.000029
16,High,1,4,Archaea,Thaumarchaeota,,,0.000004
17,High,1,4,Bacteria,,,,0.020489
18,High,1,4,Bacteria,Acidobacteria,,,0.000041
21,High,1,4,Bacteria,Actinobacteria,,,0.000009
22,High,1,4,Bacteria,Actinobacteria,Actinobacteria,,0.000062
24,High,1,4,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,0.008812


In [21]:
rd_order.shape

(3107, 8)

In [22]:
loaded_data.shape

(64755, 10)

In [23]:
rd_family = bacteriopop_utils.reduce_data(dataframe = loaded_data, 
                                          min_abundance = 0.05,
                                          phylo_column='family',
                                          #oxygen='low'
                                          )
print rd_family.shape
rd_family.head()

all groups of oxygen/week/replicate have abundances that sum to 1
columns after aggregating on phylo level: Index([u'oxygen', u'replicate', u'week', u'kingdom', u'phylum', u'class',
       u'order', u'family', u'abundance'],
      dtype='object')
first (up to) 5 phylo columns to 
keep: ['Flavobacteriaceae' 'Comamonadaceae' 'Methylophilaceae' 'Methylococcaceae'
 '']
(5564, 9)


,oxygen,replicate,week,kingdom,phylum,class,order,family,abundance
0,High,1,4,Archaea,,,,,0.000028
1,High,1,4,Archaea,Crenarchaeota,Thermoprotei,,,0.000026
5,High,1,4,Archaea,Crenarchaeota,Thermoprotei,Thermoproteales,,0.000005
8,High,1,4,Archaea,Euryarchaeota,,,,0.000032
10,High,1,4,Archaea,Euryarchaeota,Halobacteria,,,0.000029


## `demo of break_apart_experiments()`

In [24]:
bae = bacteriopop_utils.break_apart_experiments(rd_order.reset_index())

('High', 1)
('High', 2)
('High', 3)
('High', 4)
('Low', 1)
('Low', 2)
('Low', 3)
('Low', 4)
dictionary keys: [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]


In [25]:
bae.keys()

[('High', 4),
 ('Low', 1),
 ('High', 3),
 ('Low', 2),
 ('High', 2),
 ('Low', 3),
 ('High', 1),
 ('Low', 4)]

In [26]:
bae.values()[0]

,index,oxygen,replicate,week,kingdom,phylum,class,order,abundance
1184,4026,High,4,4,Archaea,Euryarchaeota,,,0.000012
1185,4034,High,4,4,Archaea,Thaumarchaeota,,,0.000010
1186,4035,High,4,4,Bacteria,,,,0.023130
1187,4036,High,4,4,Bacteria,Acidobacteria,,,0.000063
1188,4040,High,4,4,Bacteria,Actinobacteria,,,0.000004
1189,4041,High,4,4,Bacteria,Actinobacteria,Actinobacteria,,0.000005
1190,4043,High,4,4,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,0.002334
1191,4048,High,4,4,Bacteria,Aquificae,Aquificae,,0.000008
1192,4050,High,4,4,Bacteria,Armatimonadetes,,,0.000017
1193,4052,High,4,4,Bacteria,Bacteroidetes,,,0.009136


In [27]:
bacteriopop_utils.pivot_for_abundance_matrix(
    bae.values()[0].reset_index())

week,4,5,6,7,8,9,10,11,12,13,14
phylo_concat,,,,,,,,,,,
"Archaea,,,",NaN,0.000018,0.000004,NaN,NaN,0.000094,NaN,1.200000e-06,NaN,NaN,NaN
"Archaea,Crenarchaeota,,",NaN,0.000008,NaN,0.000002,NaN,0.000012,NaN,NaN,NaN,NaN,NaN
"Archaea,Crenarchaeota,Thermoprotei,",NaN,0.000007,NaN,NaN,NaN,0.000030,NaN,NaN,NaN,NaN,NaN
"Archaea,Euryarchaeota,,",0.000012,0.000019,0.000006,0.000005,0.000005,0.000138,3.450000e-06,1.200000e-06,NaN,0.000003,NaN
"Archaea,Euryarchaeota,Halobacteria,",NaN,NaN,NaN,NaN,NaN,0.000004,NaN,NaN,NaN,NaN,NaN
"Archaea,Euryarchaeota,Methanomicrobia,",NaN,0.000015,0.000003,NaN,0.000002,0.000014,6.540000e-07,1.540000e-06,NaN,NaN,0.000003
"Archaea,Thaumarchaeota,,",0.000010,0.000023,NaN,NaN,NaN,0.000047,NaN,NaN,NaN,0.000002,0.000002
"Bacteria,,,",0.023130,0.013129,0.007104,0.008717,0.008218,0.012571,5.797073e-03,4.896957e-03,0.004307,0.003573,0.002712
"Bacteria,Acidobacteria,,",0.000063,0.000050,0.000010,0.000023,0.000070,0.000114,1.220000e-05,3.030000e-05,0.000005,0.000004,0.000004


In [28]:
bae.values()[0].columns

Index([u'index', u'oxygen', u'replicate', u'week', u'kingdom', u'phylum',
       u'class', u'order', u'abundance'],
      dtype='object')

In [29]:
bacteriopop_utils.pivot_for_abundance_matrix(bae.values()[0])

week,4,5,6,7,8,9,10,11,12,13,14
phylo_concat,,,,,,,,,,,
"Archaea,,,",NaN,0.000018,0.000004,NaN,NaN,0.000094,NaN,1.200000e-06,NaN,NaN,NaN
"Archaea,Crenarchaeota,,",NaN,0.000008,NaN,0.000002,NaN,0.000012,NaN,NaN,NaN,NaN,NaN
"Archaea,Crenarchaeota,Thermoprotei,",NaN,0.000007,NaN,NaN,NaN,0.000030,NaN,NaN,NaN,NaN,NaN
"Archaea,Euryarchaeota,,",0.000012,0.000019,0.000006,0.000005,0.000005,0.000138,3.450000e-06,1.200000e-06,NaN,0.000003,NaN
"Archaea,Euryarchaeota,Halobacteria,",NaN,NaN,NaN,NaN,NaN,0.000004,NaN,NaN,NaN,NaN,NaN
"Archaea,Euryarchaeota,Methanomicrobia,",NaN,0.000015,0.000003,NaN,0.000002,0.000014,6.540000e-07,1.540000e-06,NaN,NaN,0.000003
"Archaea,Thaumarchaeota,,",0.000010,0.000023,NaN,NaN,NaN,0.000047,NaN,NaN,NaN,0.000002,0.000002
"Bacteria,,,",0.023130,0.013129,0.007104,0.008717,0.008218,0.012571,5.797073e-03,4.896957e-03,0.004307,0.003573,0.002712
"Bacteria,Acidobacteria,,",0.000063,0.000050,0.000010,0.000023,0.000070,0.000114,1.220000e-05,3.030000e-05,0.000005,0.000004,0.000004


In [30]:
import pandas as pd
toy = {1:'a', 2:'b', 3:'c'}
for key, value in toy.items():
    print key, value
    value = toy[key] = pd.DataFrame([{'z':1, 'abc':'d'}, {'z':3, 'abc':'dx'}])
for v in toy.values():
    print v

1 a
2 b
3 c
  abc  z
0   d  1
1  dx  3
  abc  z
0   d  1
1  dx  3
  abc  z
0   d  1
1  dx  3


In [31]:
prepared_DMD_matrices = bacteriopop_utils.prepare_DMD_matrices(
    min_abundance=0.01, phylo_column='order', oxygen='all')

all groups of oxygen/week/replicate have abundances that sum to 1
columns after aggregating on phylo level: Index([u'oxygen', u'replicate', u'week', u'kingdom', u'phylum', u'class',
       u'order', u'abundance'],
      dtype='object')
first (up to) 5 phylo columns to 
keep: ['' 'Flavobacteriales' 'Rhodobacterales' 'Burkholderiales'
 'Methylophilales']
('High', 1)
('High', 2)
('High', 3)
('High', 4)
('Low', 1)
('Low', 2)
('Low', 3)
('Low', 4)
dictionary keys: [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]
dataframe_dict.keys(): [('High', 4), ('Low', 1), ('High', 3), ('Low', 2), ('High', 2), ('Low', 3), ('High', 1), ('Low', 4)]
week                                       4         5         6         7   \
phylo_concat                                                                  
Archaea,,,                           0.000000  0.000018  0.000004  0.000000   
Archaea,Crenarchaeota,,              0.000000  0.000008  0.000000  0.00000

In [32]:
print "woo hoo!"

woo hoo!


In [33]:
prepared_DMD_matrices.values()[0].head()

week,4,5,6,7,8,9,10,11,12,13,14
phylo_concat,,,,,,,,,,,
"Archaea,,,",0.000000,0.000018,0.000004,0.000000,0.000000,0.000094,0.000000,0.000001,0,0.000000,0
"Archaea,Crenarchaeota,,",0.000000,0.000008,0.000000,0.000002,0.000000,0.000012,0.000000,0.000000,0,0.000000,0
"Archaea,Crenarchaeota,Thermoprotei,",0.000000,0.000007,0.000000,0.000000,0.000000,0.000030,0.000000,0.000000,0,0.000000,0
"Archaea,Euryarchaeota,,",0.000012,0.000019,0.000006,0.000005,0.000005,0.000138,0.000003,0.000001,0,0.000003,0
"Archaea,Euryarchaeota,Halobacteria,",0.000000,0.000000,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0,0.000000,0


In [34]:
prepared_DMD_matrices.values()[0].fillna(0, inplace=True)

In [35]:
prepared_DMD_matrices.values()[0]

week,4,5,6,7,8,9,10,11,12,13,14
phylo_concat,,,,,,,,,,,
"Archaea,,,",0.000000,0.000018,0.000004,0.000000,0.000000,0.000094,0.000000e+00,1.200000e-06,0.000000,0.000000,0.000000
"Archaea,Crenarchaeota,,",0.000000,0.000008,0.000000,0.000002,0.000000,0.000012,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
"Archaea,Crenarchaeota,Thermoprotei,",0.000000,0.000007,0.000000,0.000000,0.000000,0.000030,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
"Archaea,Euryarchaeota,,",0.000012,0.000019,0.000006,0.000005,0.000005,0.000138,3.450000e-06,1.200000e-06,0.000000,0.000003,0.000000
"Archaea,Euryarchaeota,Halobacteria,",0.000000,0.000000,0.000000,0.000000,0.000000,0.000004,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
"Archaea,Euryarchaeota,Methanomicrobia,",0.000000,0.000015,0.000003,0.000000,0.000002,0.000014,6.540000e-07,1.540000e-06,0.000000,0.000000,0.000003
"Archaea,Thaumarchaeota,,",0.000010,0.000023,0.000000,0.000000,0.000000,0.000047,0.000000e+00,0.000000e+00,0.000000,0.000002,0.000002
"Bacteria,,,",0.023130,0.013129,0.007104,0.008717,0.008218,0.012571,5.797073e-03,4.896957e-03,0.004307,0.003573,0.002712
"Bacteria,Acidobacteria,,",0.000063,0.000050,0.000010,0.000023,0.000070,0.000114,1.220000e-05,3.030000e-05,0.000005,0.000004,0.000004
